In [ ]:
!pip install spuco

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s

In [ ]:
import torch
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T
from spuco.evaluate import Evaluator
from spuco.models import model_factory
from spuco.utils import Trainer
from torch.optim import SGD
from spuco.evaluate import Evaluator
from spuco.group_inference import Cluster, ClusterAlg
from spuco.robust_train import GroupBalanceBatchERM


In [ ]:
device = torch.device("cuda")

In [ ]:
classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train"
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

100%|██████████| 9.91M/9.91M [00:00<00:00, 16.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 494kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.47MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.85MB/s]


In [ ]:
model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
trainer = Trainer(
    trainset=trainset,
    model=model,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-3, weight_decay=5e-4, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)

trainer.train(1)

Epoch 0:   2%|▏         | 13/751 [00:00<00:18, 39.21batch/s, accuracy=14.0625%, loss=1.61]

 | Epoch 0 | Loss: 1.6119283437728882 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6037821769714355 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6132866144180298 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.614161491394043 | Accuracy: 14.0625%
 | Epoch 0 | Loss: 1.6157965660095215 | Accuracy: 6.25%
 | Epoch 0 | Loss: 1.611609697341919 | Accuracy: 14.0625%
 | Epoch 0 | Loss: 1.6173245906829834 | Accuracy: 6.25%
 | Epoch 0 | Loss: 1.608636736869812 | Accuracy: 9.375%
 | Epoch 0 | Loss: 1.6143556833267212 | Accuracy: 6.25%
 | Epoch 0 | Loss: 1.605649471282959 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6209099292755127 | Accuracy: 14.0625%
 | Epoch 0 | Loss: 1.614650845527649 | Accuracy: 10.9375%
 | Epoch 0 | Loss: 1.600382685661316 | Accuracy: 10.9375%
 | Epoch 0 | Loss: 1.5990301370620728 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.625059723854065 | Accuracy: 7.8125%
 | Epoch 0 | Loss: 1.6021331548690796 | Accuracy: 9.375%
 | Epoch 0 | Loss: 1.614708423614502 | Accuracy: 4.6875%
 | Epoch 0 | Loss: 1.61143767

Epoch 0:   6%|▋         | 47/751 [00:00<00:06, 104.16batch/s, accuracy=21.875%, loss=1.61]

 | Epoch 0 | Loss: 1.6112143993377686 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.6028881072998047 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.6036810874938965 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.603493094444275 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.609642505645752 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6081331968307495 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.6230287551879883 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.6068886518478394 | Accuracy: 10.9375%
 | Epoch 0 | Loss: 1.6088228225708008 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6129179000854492 | Accuracy: 10.9375%
 | Epoch 0 | Loss: 1.611122727394104 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.6151375770568848 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.6114840507507324 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6109051704406738 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.6014841794967651 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.598789095878601 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6031310558319092 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.617

Epoch 0:  10%|█         | 78/751 [00:00<00:05, 126.30batch/s, accuracy=28.125%, loss=1.6]

...

Epoch 0:  96%|█████████▌| 720/751 [00:05<00:00, 146.96batch/s, accuracy=100.0%, loss=0.0489]  

 | Epoch 0 | Loss: 0.26831570267677307 | Accuracy: 96.875%
 | Epoch 0 | Loss: 0.12757781147956848 | Accuracy: 95.3125%
 | Epoch 0 | Loss: 0.1215502917766571 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.0834166631102562 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.11224530637264252 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.08227185904979706 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.09116815775632858 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.15091417729854584 | Accuracy: 96.875%
 | Epoch 0 | Loss: 0.07348126918077469 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.10019044578075409 | Accuracy: 96.875%
 | Epoch 0 | Loss: 0.08708421140909195 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.06429512798786163 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.13865700364112854 | Accuracy: 95.3125%
 | Epoch 0 | Loss: 0.07918865978717804 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.09565252810716629 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.2589854896068573 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.10547400265932083 | Accuracy:

Epoch 0:  98%|█████████▊| 736/751 [00:05<00:00, 148.58batch/s, accuracy=100.0%, loss=0.0476]

 | Epoch 0 | Loss: 0.08206894993782043 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.0745982676744461 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.06030154228210449 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.07119908928871155 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.34295883774757385 | Accuracy: 95.3125%
 | Epoch 0 | Loss: 0.04289179667830467 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.14132876694202423 | Accuracy: 96.875%
 | Epoch 0 | Loss: 0.052680984139442444 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.40750157833099365 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.08178649842739105 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.05857900157570839 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.07984298467636108 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.07747936248779297 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.0867486447095871 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.13221874833106995 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.10767749696969986 | Accuracy: 98.4375%
 | Epoch 0 | Loss: 0.0761648565530777 | Accuracy: 96

Epoch 0: 100%|██████████| 751/751 [00:05<00:00, 142.71batch/s, accuracy=100.0%, loss=0.0476]


In [ ]:
evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:07,  3.16it/s]

Group (0, 0) Accuracy: 97.39952718676123


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:07,  3.17it/s]

Group (0, 1) Accuracy: 0.0


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:00<00:07,  3.13it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:08<00:00,  3.15it/s]

Group (4, 3) Accuracy: 0.0


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:08<00:00,  2.81it/s]

Group (4, 4) Accuracy: 100.0


{(0, 0): 97.39952718676123,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 2.127659574468085,
 (1, 0): 0.0,
 (1, 1): 100.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 98.66666666666667,
 (2, 3): 1.6,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 3.27455919395466,
 (3, 3): 96.97732997481108,
 (3, 4): 0.0,
 (4, 0): 2.2670025188916876,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 100.0}

In [ ]:
import spuco

print(spuco.__file__)

/usr/local/lib/python3.11/dist-packages/spuco/__init__.py


In [ ]:
# %load /usr/local/lib/python3.11/dist-packages/spuco/utils/exemplar_cluster.py
from typing import Dict, List

import numpy as np
from tqdm import tqdm

from spuco.utils.submodular import FacilityLocation, lazy_greedy

def cluster_by_exemplars(similarity_matrix, num_exemplars, verbose=False) -> Dict[int, List[int]]:
    """
    Returns a dictionary mapping exemplar index to a list of indices.

    :param similarity_matrix: The similarity matrix.
    :type similarity_matrix: numpy.ndarray
    :param num_exemplars: The number of exemplars to select.
    :type num_exemplars: int
    :param verbose: Whether to print progress information.
    :type verbose: bool, optional
    :return: A dictionary mapping exemplar index to a list of indices.
    :rtype: dict[int, list[int]]]
    """
    submodular_function = FacilityLocation(D=similarity_matrix, V=range(len(similarity_matrix)))
    exemplar_indices, _ = lazy_greedy(F=submodular_function, V=range(len(similarity_matrix)), B=num_exemplars, verbose=verbose)
    clusters = {}

    for exemplar_index in exemplar_indices:
        clusters[exemplar_index] = []

    for index in tqdm(range(len(similarity_matrix)), desc="Sorting samples by exemplar", disable=not verbose):
        exemplar_index, _  = closest_exemplar(index, exemplar_indices, similarity_matrix)
        clusters[exemplar_index].append(index)

    return clusters

def closest_exemplar(sample_index, exemplar_indices, similarity_matrix):
    """
    Finds the closest exemplar to a given sample index.

    :param sample_index: The index of the sample.
    :type sample_index: int
    :param exemplar_indices: The indices of the exemplars.
    :type exemplar_indices: list[int]
    :param similarity_matrix: The similarity matrix.
    :type similarity_matrix: numpy.ndarray
    :return: The index of the closest exemplar and the similarity score.
    :rtype: tuple[int, float]
    """

    max_similarity = -np.inf
    best_exemplar_index = -1

    for curr_exemplar_index in exemplar_indices:
        if similarity_matrix[sample_index][curr_exemplar_index] > max_similarity:
            max_similarity = similarity_matrix[sample_index][curr_exemplar_index]
            best_exemplar_index = curr_exemplar_index

    return best_exemplar_index, max_similarity


In [ ]:
# this cell needs to be ran to fix np.inf compatability issue
%%writefile /usr/local/lib/python3.11/dist-packages/spuco/utils/exemplar_cluster.py
from typing import Dict, List

import numpy as np
from tqdm import tqdm

from spuco.utils.submodular import FacilityLocation, lazy_greedy

def cluster_by_exemplars(similarity_matrix, num_exemplars, verbose=False) -> Dict[int, List[int]]:
    """
    Returns a dictionary mapping exemplar index to a list of indices.

    :param similarity_matrix: The similarity matrix.
    :type similarity_matrix: numpy.ndarray
    :param num_exemplars: The number of exemplars to select.
    :type num_exemplars: int
    :param verbose: Whether to print progress information.
    :type verbose: bool, optional
    :return: A dictionary mapping exemplar index to a list of indices.
    :rtype: dict[int, list[int]]]
    """
    submodular_function = FacilityLocation(D=similarity_matrix, V=range(len(similarity_matrix)))
    exemplar_indices, _ = lazy_greedy(F=submodular_function, V=range(len(similarity_matrix)), B=num_exemplars, verbose=verbose)
    clusters = {}

    for exemplar_index in exemplar_indices:
        clusters[exemplar_index] = []

    for index in tqdm(range(len(similarity_matrix)), desc="Sorting samples by exemplar", disable=not verbose):
        exemplar_index, _  = closest_exemplar(index, exemplar_indices, similarity_matrix)
        clusters[exemplar_index].append(index)

    return clusters

def closest_exemplar(sample_index, exemplar_indices, similarity_matrix):
    """
    Finds the closest exemplar to a given sample index.

    :param sample_index: The index of the sample.
    :type sample_index: int
    :param exemplar_indices: The indices of the exemplars.
    :type exemplar_indices: list[int]
    :param similarity_matrix: The similarity matrix.
    :type similarity_matrix: numpy.ndarray
    :return: The index of the closest exemplar and the similarity score.
    :rtype: tuple[int, float]
    """

    max_similarity = -np.inf
    best_exemplar_index = -1

    for curr_exemplar_index in exemplar_indices:
        if similarity_matrix[sample_index][curr_exemplar_index] > max_similarity:
            max_similarity = similarity_matrix[sample_index][curr_exemplar_index]
            best_exemplar_index = curr_exemplar_index

    return best_exemplar_index, max_similarity


Overwriting /usr/local/lib/python3.11/dist-packages/spuco/utils/exemplar_cluster.py


In [ ]:
import importlib
import spuco
importlib.invalidate_caches()
importlib.reload(spuco.utils.exemplar_cluster)
importlib.reload(spuco)

<module 'spuco' from '/usr/local/lib/python3.11/dist-packages/spuco/__init__.py'>

In [ ]:
import importlib
importlib.reload(spuco)
outputs = trainer.get_trainset_outputs()
cluster = Cluster(
    Z=outputs,
    class_labels=trainset.labels,
    cluster_alg=ClusterAlg.KMEANS,
    num_clusters=2,
    device=device,
    verbose=True
)
group_partition = cluster.infer_groups()

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Clustering class-wise: 100%|██████████| 5/5 [00:00<00:00,  6.16it/s]


In [ ]:
for key in sorted(group_partition.keys()):
    print(key, len(group_partition[key]))

(0, 0) 10093
(0, 1) 40
(1, 0) 9645
(1, 1) 27
(2, 0) 8971
(2, 1) 40
(3, 0) 9723
(3, 1) 24
(4, 0) 9400
(4, 1) 41


In [ ]:
evaluator = Evaluator(
    testset=trainset,
    group_partition=group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Evaluating group-wise accuracy:  10%|█         | 1/10 [00:01<00:14,  1.61s/it]

Group (0, 0) Accuracy: 97.81036361834936


Evaluating group-wise accuracy:  20%|██        | 2/10 [00:02<00:07,  1.11it/s]

Group (0, 1) Accuracy: 0.0


Evaluating group-wise accuracy:  30%|███       | 3/10 [00:03<00:07,  1.05s/it]

...

Evaluating group-wise accuracy:  90%|█████████ | 9/10 [00:06<00:00,  1.50it/s]

Group (4, 0) Accuracy: 99.93617021276596


Evaluating group-wise accuracy: 100%|██████████| 10/10 [00:07<00:00,  1.36it/s]

Group (4, 1) Accuracy: 0.0


{(0, 0): 97.81036361834936,
 (0, 1): 0.0,
 (1, 0): 99.77190254017626,
 (1, 1): 0.0,
 (2, 0): 98.87415003901461,
 (2, 1): 0.0,
 (3, 0): 98.16928931399774,
 (3, 1): 0.0,
 (4, 0): 99.93617021276596,
 (4, 1): 0.0}

In [ ]:
better_model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
group_balance_erm = GroupBalanceBatchERM(
    model=better_model,
    num_epochs=10,
    trainset=trainset,
    group_partition=group_partition,
    batch_size=64,
    optimizer=SGD(better_model.parameters(),  lr=1e-3, weight_decay=5e-4, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
group_balance_erm.train()

Epoch 0:   2%|▏         | 14/751 [00:00<00:15, 46.99batch/s, accuracy=7.8125%, loss=1.63] 

GB | Epoch 0 | Loss: 1.616376280784607 | Accuracy: 18.75%
GB | Epoch 0 | Loss: 1.605672836303711 | Accuracy: 15.625%
GB | Epoch 0 | Loss: 1.6066949367523193 | Accuracy: 18.75%
GB | Epoch 0 | Loss: 1.6147258281707764 | Accuracy: 17.1875%
GB | Epoch 0 | Loss: 1.6161426305770874 | Accuracy: 20.3125%
GB | Epoch 0 | Loss: 1.6099820137023926 | Accuracy: 14.0625%
GB | Epoch 0 | Loss: 1.6053920984268188 | Accuracy: 23.4375%
GB | Epoch 0 | Loss: 1.6124390363693237 | Accuracy: 18.75%
GB | Epoch 0 | Loss: 1.6179320812225342 | Accuracy: 17.1875%
GB | Epoch 0 | Loss: 1.6133649349212646 | Accuracy: 20.3125%
GB | Epoch 0 | Loss: 1.6063042879104614 | Accuracy: 26.5625%
GB | Epoch 0 | Loss: 1.6189731359481812 | Accuracy: 14.0625%
GB | Epoch 0 | Loss: 1.6170413494110107 | Accuracy: 15.625%
GB | Epoch 0 | Loss: 1.608757734298706 | Accuracy: 23.4375%
GB | Epoch 0 | Loss: 1.6066327095031738 | Accuracy: 21.875%
GB | Epoch 0 | Loss: 1.630560278892517 | Accuracy: 12.5%
GB | Epoch 0 | Loss: 1.6152048110961914 

Epoch 0:   6%|▋         | 47/751 [00:00<00:06, 109.07batch/s, accuracy=29.6875%, loss=1.61]

GB | Epoch 0 | Loss: 1.6103172302246094 | Accuracy: 17.1875%
GB | Epoch 0 | Loss: 1.6156730651855469 | Accuracy: 18.75%
GB | Epoch 0 | Loss: 1.611386775970459 | Accuracy: 18.75%
GB | Epoch 0 | Loss: 1.6055184602737427 | Accuracy: 21.875%
GB | Epoch 0 | Loss: 1.6248241662979126 | Accuracy: 18.75%
GB | Epoch 0 | Loss: 1.6120421886444092 | Accuracy: 25.0%
GB | Epoch 0 | Loss: 1.606407880783081 | Accuracy: 31.25%
GB | Epoch 0 | Loss: 1.6120516061782837 | Accuracy: 26.5625%
GB | Epoch 0 | Loss: 1.6072014570236206 | Accuracy: 26.5625%
GB | Epoch 0 | Loss: 1.6101902723312378 | Accuracy: 21.875%
GB | Epoch 0 | Loss: 1.60452401638031 | Accuracy: 26.5625%
GB | Epoch 0 | Loss: 1.602498173713684 | Accuracy: 26.5625%
GB | Epoch 0 | Loss: 1.606472134590149 | Accuracy: 28.125%
GB | Epoch 0 | Loss: 1.606865406036377 | Accuracy: 17.1875%
GB | Epoch 0 | Loss: 1.603924036026001 | Accuracy: 25.0%
GB | Epoch 0 | Loss: 1.6113475561141968 | Accuracy: 12.5%
GB | Epoch 0 | Loss: 1.6119147539138794 | Accuracy: 

Epoch 0:  11%|█         | 79/751 [00:00<00:05, 133.55batch/s, accuracy=17.1875%, loss=1.61]

...

Epoch 9:  98%|█████████▊| 736/751 [00:07<00:00, 106.25batch/s, accuracy=98.4375%, loss=0.0273]

GB | Epoch 9 | Loss: 0.02006799913942814 | Accuracy: 100.0%
GB | Epoch 9 | Loss: 0.01907799020409584 | Accuracy: 100.0%
GB | Epoch 9 | Loss: 0.014814738184213638 | Accuracy: 100.0%
GB | Epoch 9 | Loss: 0.03372849524021149 | Accuracy: 98.4375%
GB | Epoch 9 | Loss: 0.034012097865343094 | Accuracy: 98.4375%
GB | Epoch 9 | Loss: 0.019365200772881508 | Accuracy: 98.4375%
GB | Epoch 9 | Loss: 0.049725137650966644 | Accuracy: 98.4375%
GB | Epoch 9 | Loss: 0.06334876269102097 | Accuracy: 98.4375%
GB | Epoch 9 | Loss: 0.03791162371635437 | Accuracy: 98.4375%
GB | Epoch 9 | Loss: 0.012049587443470955 | Accuracy: 100.0%
GB | Epoch 9 | Loss: 0.015672633424401283 | Accuracy: 100.0%
GB | Epoch 9 | Loss: 0.011830687522888184 | Accuracy: 100.0%
GB | Epoch 9 | Loss: 0.013821830041706562 | Accuracy: 100.0%
GB | Epoch 9 | Loss: 0.04641519859433174 | Accuracy: 98.4375%
GB | Epoch 9 | Loss: 0.04960229992866516 | Accuracy: 98.4375%
GB | Epoch 9 | Loss: 0.008782165125012398 | Accuracy: 100.0%
GB | Epoch 9 | 

Epoch 9: 100%|██████████| 751/751 [00:07<00:00, 118.04batch/s, accuracy=100.0%, loss=0.000206]

GB | Epoch 9 | Loss: 0.016706187278032303 | Accuracy: 100.0%
GB | Epoch 9 | Loss: 0.023577222600579262 | Accuracy: 98.4375%
GB | Epoch 9 | Loss: 0.013218635693192482 | Accuracy: 100.0%
GB | Epoch 9 | Loss: 0.015474628657102585 | Accuracy: 100.0%
GB | Epoch 9 | Loss: 0.017495552077889442 | Accuracy: 100.0%
GB | Epoch 9 | Loss: 0.014825286343693733 | Accuracy: 100.0%
GB | Epoch 9 | Loss: 0.18921130895614624 | Accuracy: 98.4375%
GB | Epoch 9 | Loss: 0.006940525956451893 | Accuracy: 100.0%
GB | Epoch 9 | Loss: 0.026518158614635468 | Accuracy: 100.0%
GB | Epoch 9 | Loss: 0.011062962003052235 | Accuracy: 100.0%
GB | Epoch 9 | Loss: 0.01645934395492077 | Accuracy: 100.0%
GB | Epoch 9 | Loss: 0.00020637200213968754 | Accuracy: 100.0%


Epoch 9: 100%|██████████| 751/751 [00:07<00:00, 95.43batch/s, accuracy=100.0%, loss=0.000206] 


In [ ]:
evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=better_model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:09,  2.61it/s]

Group (0, 0) Accuracy: 99.76359338061465


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:08,  2.60it/s]

Group (0, 1) Accuracy: 87.23404255319149


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:01<00:08,  2.51it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:10<00:00,  2.41it/s]

Group (4, 3) Accuracy: 73.73737373737374


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:10<00:00,  2.35it/s]

Group (4, 4) Accuracy: 97.22222222222223


{(0, 0): 99.76359338061465,
 (0, 1): 87.23404255319149,
 (0, 2): 55.082742316784866,
 (0, 3): 84.39716312056737,
 (0, 4): 8.51063829787234,
 (1, 0): 55.256723716381416,
 (1, 1): 98.77750611246944,
 (1, 2): 37.990196078431374,
 (1, 3): 25.735294117647058,
 (1, 4): 75.0,
 (2, 0): 71.46666666666667,
 (2, 1): 61.86666666666667,
 (2, 2): 99.2,
 (2, 3): 9.6,
 (2, 4): 63.101604278074866,
 (3, 0): 75.12562814070351,
 (3, 1): 5.289672544080605,
 (3, 2): 1.7632241813602014,
 (3, 3): 99.74811083123426,
 (3, 4): 70.27707808564232,
 (4, 0): 11.335012594458439,
 (4, 1): 53.40050377833753,
 (4, 2): 63.97984886649874,
 (4, 3): 73.73737373737374,
 (4, 4): 97.22222222222223}